In [1]:
import numpy as np
import keras
from keras.preprocessing import text, sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from keras.callbacks import TensorBoard, ModelCheckpoint

Using TensorFlow backend.


In [2]:
fname = "data/alice.txt"
raw_text = open(fname).read().lower()

In [3]:
text_len = len(raw_text)
print('Text length:', text_len)

Text length: 163817


In [4]:
charset = sorted(list(set(raw_text)))
char_to_int = {c:i for (i,c) in enumerate(charset)}
int_to_char = {i:c for (i,c) in enumerate(charset)}
n_vocab = len(charset)
print('Number of characters:', n_vocab)

Number of characters: 61


In [5]:
max_seq_len = 100

In [6]:
seqs, next_chars = [], []
for i in range(text_len - max_seq_len):
    seqs.append(raw_text[i:i+max_seq_len])
    next_chars.append(raw_text[i+max_seq_len])
n_seqs = len(seqs)
print('Number of sequences:', n_seqs)

Number of sequences: 163717


In [7]:
X = np.zeros([n_seqs, max_seq_len, n_vocab], dtype=np.bool)
y = np.zeros([n_seqs, n_vocab], dtype=np.bool)

for i,seq in enumerate(seqs):
    for j,c in enumerate(seq):
        X[i,j,char_to_int[c]] = 1
    y[i,char_to_int[next_chars[i]]] = 1

In [8]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [9]:
model = Sequential()
model.add(LSTM(256, input_shape=(max_seq_len,n_vocab)))
model.add(Dropout(0.2))
model.add(Dense(n_vocab, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [10]:
model_checkpoint = ModelCheckpoint('weights.{epoch:02d}-{loss:.2f}.hdf5', monitor='loss', verbose=0,
                                  save_best_only=True, mode='min')
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=False)
callbacks_list = [model_checkpoint, tensorboard]

In [11]:
# Uncomment to retrain

# nb_epoch = 20
# batch_size = 128
# model.fit(X, y, nb_epoch=nb_epoch, batch_size=batch_size, callbacks=callbacks_list)

In [12]:
# Loading model from weight

weight_fname = 'alice_weights_iter20_loss1.10.hdf5'
model.load_weights(weight_fname)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [18]:
# Generate random start

num_iter = 1000
temperature = 0.8
start = np.random.randint(0,text_len-max_seq_len-1,size=1)
sentence = raw_text[start:start+max_seq_len]
print('\n---Starting sentence:---\n')
print(sentence)


print('\n---Generated text:---\n')
for i in range(num_iter):
    x = np.zeros((1, max_seq_len, n_vocab))
    for j,c in enumerate(sentence):
        x[0,j,char_to_int[c]] = 1
    prediction = model.predict(x)[0]
    sample_idx = int(sample(prediction,temperature))
    next_char = int_to_char[sample_idx]
    sentence = sentence[1:] + next_char
    print(next_char,end='')
    

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:6: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future



---Starting sentence:---


‘oh, as to the whiting,’ said the mock turtle, ‘they--you’ve seen them,
of course?’

‘yes,’ said al

---Generated text:---

ice, and a long all to the same weak in it, ‘i cool found here, sthermy, and that’s she was a count--in a dry again!’ he didnnot leaged, and alice was not a little come sorpimar
tones and was going and fingea pully
wearnou in and off, and de, when it was a duchess to see the use of the offore that, and she heard
at the shill oner bifte of the hadds.

‘out fer ith of rath hep a sound--everything i’ve bedid,’ said the hatter in the
good.

‘i don’t say a meaning
a doess in that in
the simp of the court!’


the hatter with a samply: ‘beciatafile,’ he thought, ‘i’ll get out it pursand it to do
one off a time: it’s no one of the table
word election or use and amary was which was going and much of it was till as she could go,
in the
other play with
an a procosside. there were see it to shoulder at the queen off the sime, and against one of the